In [63]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
import pickle
import dataframe_image as dfi

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/mkucz95/kickstarter_data/master/ks-projects-201801.csv')

# IA. Exploratory Data Analysis

In [14]:
# report = ProfileReport(df)
# report

# IB. Wrangle Data/Feature Engineering

In [15]:
# drop the ID , name , category(sub_category) , usd_pledged , usd_pledged_real ,usd_goal_real
df = df[['main_category','launched','deadline','currency','goal','pledged','backers','state']]

In [16]:
# turned the deadline and launched columns to datetime objects
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])

In [17]:
# subtracted the deadline from the launched to get the campaign days and created a new feature campaign_length
df['campaign_length'] = df['deadline'] - df['launched']
df['campaign_length'] = df['campaign_length'].dt.days

In [18]:
# main_category is the only category I will be concerned with
df = df.rename(columns={'main_category': 'category'})

In [19]:
df['percentage_pledged'] = (df['pledged'] / df['goal']) * 100
# divided the pledged and goal and multiplied by 100 
df = df[['category','campaign_length','currency','percentage_pledged','backers','state']]

# drop the live and undefined rows , group the cancelled and suspended states to failed
df.drop(df[(df.state == 'live')].index,inplace=True)
df.drop(df[(df.state == 'undefined')].index,inplace=True)
df.loc[df.state == "canceled", 'state'] = "failed"
df.loc[df.state == "suspended", 'state'] = "failed"

# # create dummy variables of currencies 
# currencies=pd.get_dummies(df.currency,prefix='currency')
# df=pd.concat([df,currencies],axis=1)
# df.drop('currency', axis=1, inplace=True)

# # create dummy variables of categories
# categories=pd.get_dummies(df.category,prefix='category')
# df=pd.concat([df,categories],axis=1)
# df.drop('category', axis=1, inplace=True)

# # create dummy variables of states 
# states=pd.get_dummies(df.state,drop_first=True)
# df=pd.concat([df,states],axis=1)
# df.drop('state',axis=1,inplace=True)

In [66]:
test_art=df[df['category']=='Art']

In [71]:
example=test_art[['percentage_pledged','state']]
testing = example[example['percentage_pledged'] >= 100] 

In [73]:
testing.state.value_counts()

successful    11508
failed           52
Name: state, dtype: int64

In [75]:
testing_2 =  example[example['percentage_pledged'] < 100] 

In [77]:
testing_2.state.value_counts()

failed        16397
successful        2
Name: state, dtype: int64

In [68]:
# dfi.export(example, 'dataframe.png')
example.to_csv('example_Art.csv', index=False)

# II. Split Data

In [10]:
target = 'successful'
y = df[target]
X = df.drop(columns=target)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=.6, test_size=.4, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,train_size=.5,test_size=.5, random_state=1)

# III. Establish Baseline

In [12]:
y_train.value_counts(normalize=True).max()

0.6408317664965529

In [13]:
print(df.columns)

Index(['campaign_length', 'percentage_pledged', 'backers', 'currency_AUD',
       'currency_CAD', 'currency_CHF', 'currency_DKK', 'currency_EUR',
       'currency_GBP', 'currency_HKD', 'currency_JPY', 'currency_MXN',
       'currency_NOK', 'currency_NZD', 'currency_SEK', 'currency_SGD',
       'currency_USD', 'category_Art', 'category_Comics', 'category_Crafts',
       'category_Dance', 'category_Design', 'category_Fashion',
       'category_Film & Video', 'category_Food', 'category_Games',
       'category_Journalism', 'category_Music', 'category_Photography',
       'category_Publishing', 'category_Technology', 'category_Theater',
       'successful'],
      dtype='object')


,campaign_length,percentage_pledged,backers,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,currency_HKD,...,category_Film & Video,category_Food,category_Games,category_Journalism,category_Music,category_Photography,category_Publishing,category_Technology,category_Theater,successful
0,58,0.000000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,59,8.070000,15,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,44,0.488889,3,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,29,0.020000,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,55,6.579487,14,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,29,0.050000,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
378657,26,10.333333,5,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
378658,45,0.133333,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
378659,30,1.333333,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# IV. Build Model

In [24]:
# model = RandomForestClassifier()
# model.fit(X_train,y_train)

In [17]:
model_lr = LogisticRegression()
model_lr.fit(X_train,y_train)

/Users/rogervieira/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:

list_currencies = ['AUD','CAD', 'CHF', 'DKK', 'EUR','GBP', 'HKD', 
                 'JPY', 'MXN','NOK','NZD', 'SEK', 'SGD','USD']
list_categories = ['Art', 'Comics', 'Crafts','Dance', 'Design','Fashion','Film & Video',
                   'Food', 'Games','Journalism','Music', 'Photography',
                   'Publishing', 'Technology','Theater']


def currency_category_select(user_input,list_input):
    for x in range(len(list_input)):
        if list_input[x] == user_input:
            list_input[x] = 1
        else:
            list_input[x] = 0
    return list_input 


input_currency=input('Select Currency: ')
input_categories=input('Select Category: ')
input_campaign_length=input('Campaign Length: ')
input_percentage_pledged =input('percentage_pledged as int: ')
input_backers=input('Quantity of backers: ')

input_currency=currency_category_select(input_currency,list_currencies)
input_category=currency_category_select(input_categories,list_categories)

model_input = np.array([input_campaign_length,input_percentage_pledged,input_backers])
model_input = np.append(model_input,input_currency)
model_input = np.append(model_input,input_category).reshape(1, -1)

if model_lr.predict(model_input)[0] == 1:
    print('Your kickstarter is a success!')
else: 
    print('Your kickstarter failed, try again.')



Select Currency: AUD
Select Category: Art
Campaign Length: 100
percentage_pledged as int: 90
Quantity of backers: 456
Your kickstarter is a success!


/Users/rogervieira/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/rogervieira/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:561: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


In [19]:
print('Training Accuracy', model_lr.score(X_train, y_train))
print('Validation Accuracy', model_lr.score(X_val,y_val))
print('Testing Accuracy', model_lr.score(X_test, y_test))

Training Accuracy 0.8386471483570597
Validation Accuracy 0.8376980929358044
Testing Accuracy 0.8364893902766586


# V Pickle Data

In [25]:
with open('model_pickle_lr','wb') as f:
    pickle.dump(model_lr,f)

In [20]:
# with open('model_pickle_lr','rb') as f:
#     mp=pickle.load(f)